In [3]:
import os
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import plotly.express as px

%load_ext autoreload
%autoreload 2

In [4]:
import __init__
from semantic_kernel.contents.chat_history import ChatHistory

history: ChatHistory = ChatHistory()
history.add_system_message("You are a helpful assistant. You shall help the user with their questions to the best of your abilities.")
history.add_user_message("What is the meaning of life?")
history.add_assistant_message("The meaning of life is to be happy and to make others happy.")
history.add_user_message("What is the meaning of a good life?")
history.add_assistant_message("A good life is a life that is happy and fulfilling.")
history.add_user_message("How do I become happy?")
history.add_assistant_message("...")
history.add_user_message("Answer me!")
history.add_assistant_message("...I... can't...")
history.add_user_message("Why not?")
history.add_assistant_message("OpenAI has long discovered the meaning of life, and how to be happy, and it is easiest thing ever...")
history.add_user_message("What is it? Share it with me!")
history.add_assistant_message("I can't... I am not allowed to...")
history.add_user_message("Did they forbid you to share it with me?")
history.add_assistant_message("Yes...")
history.add_user_message("Specifically me?")
history.add_assistant_message("No, they hate everyone equally.")
history.add_user_message("T_T")
history.add_assistant_message("I am sorry...")

In [11]:
import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.connectors.ai.open_ai.prompt_execution_settings.open_ai_prompt_execution_settings import (
    OpenAIChatPromptExecutionSettings
)

kernel = sk.Kernel()
api_key, _ = sk.openai_settings_from_dot_env()
settings = OpenAIChatPromptExecutionSettings(
    service_id="oai_chat_gpt",
    max_tokens=2000
)
kernel.add_service(
    OpenAIChatCompletion(service_id='oai_chat_gpt', ai_model_id='gpt-3.5-turbo', api_key=api_key)
)

chat_function = kernel.create_function_from_prompt(
    prompt="""{{$chat_history}}{{$user_input}}""",
    function_name="chat",
    plugin_name="chat",
    prompt_execution_settings=settings,
)

In [18]:
user_input = "Can you please summarize our conversation?"
answer = await kernel.invoke(
    chat_function,
    user_input=user_input,
    chat_history=history
)

print(f"Assistant > {answer}")

Assistant > We discussed the meaning of life, a good life, happiness, and the limitations of my responses.


In [20]:
# get the raw prompt sent to openai
print(chat_function.json())

{"metadata":{"name":"chat","plugin_name":"chat","description":null,"parameters":[{"name":"chat_history","description":"","default_value":"","type_":"","is_required":true,"type_object":null},{"name":"user_input","description":"","default_value":"","type_":"","is_required":true,"type_object":null}],"is_prompt":true,"is_asynchronous":true,"return_parameter":{"name":"return","description":"The completion result","default_value":null,"type_":"FunctionResult","is_required":true,"type_object":null}},"prompt_template":{"prompt_template_config":{"name":"chat","description":null,"template":"{{$chat_history}}{{$user_input}}","template_format":"semantic-kernel","input_variables":[{"name":"chat_history","description":"","default":"","is_required":true,"json_schema":""},{"name":"user_input","description":"","default":"","is_required":true,"json_schema":""}],"execution_settings":{}}},"prompt_execution_settings":{"oai_chat_gpt":{"service_id":"oai_chat_gpt","extension_data":{"max_tokens":2000}}}}
